In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2

In [2]:
# Define the dimensions for your images
image_width = 128
image_height = 128
batch_size = 32

# Create an ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load training data from the 'color_image' directory
train_generator = train_datagen.flow_from_directory(
    'color_input',
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 563 images belonging to 6 classes.


In [3]:
num_classes = 6  # Replace with the actual number of classes in your dataset

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(image_width, image_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(train_generator, epochs=10, steps_per_epoch=len(train_generator), verbose=1)
model.save('models/image_classification_model.keras')

Epoch 1/10
18/18 [==============================] - 12s 618ms/step - loss: 1.8142 - accuracy: 0.1528
Epoch 2/10
18/18 [==============================] - 11s 586ms/step - loss: 1.7447 - accuracy: 0.2291
Epoch 3/10
18/18 [==============================] - 11s 604ms/step - loss: 1.6560 - accuracy: 0.2575
Epoch 4/10
18/18 [==============================] - 11s 621ms/step - loss: 1.5836 - accuracy: 0.3179
Epoch 5/10
18/18 [==============================] - 12s 686ms/step - loss: 1.4857 - accuracy: 0.3375
Epoch 6/10
18/18 [==============================] - 13s 718ms/step - loss: 1.4484 - accuracy: 0.4121
Epoch 7/10
 9/18 [==============>...............] - ETA: 6s - loss: 1.4283 - accuracy: 0.3782

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf

# Define the dimensions for input images that match your test images
image_width, image_height = 128, 128

# Load your model
model_path = 'models/image_classification_model.keras'
trained_model = tf.keras.models.load_model(model_path)

# Directory path for test images
test_directory = 'color_output'

# Create a dictionary to map class index to class name
class_names = [class_name for class_name in os.listdir(test_directory) if os.path.isdir(os.path.join(test_directory, class_name))]
class_names = sorted(class_names)
class_to_index = {class_name: index for index, class_name in enumerate(class_names)}

# Initialize variables for accuracy computation
correct_predictions = 0
total_predictions = 0

# List all subdirectories (each corresponds to a class)
class_directories = [os.path.join(test_directory, class_name) for class_name in class_names]

for class_directory in class_directories:
    class_name = os.path.basename(class_directory)
    class_index = class_to_index[class_name]
    print(f"Class: {class_name}")

    # Iterate through all images in the current class directory
    for image_filename in os.listdir(class_directory):
        if image_filename.endswith('.jpg'):
            # Preprocess the test image
            image_path = os.path.join(class_directory, image_filename)
            test_image = cv2.imread(image_path)
            test_image = cv2.resize(test_image, (image_width, image_height))
            test_image = test_image / 255.0
            test_image = np.expand_dims(test_image, axis=0)

            # Make predictions
            test_predictions = trained_model.predict(test_image)

            # Interpret the predictions
            predicted_class = np.argmax(test_predictions)

            # Check if predicted_class is within the valid range
            if predicted_class < len(class_names):
                predicted_class_name = class_names[predicted_class]
            else:
                predicted_class_name = "Unknown"

            print(f"Image: {image_filename}, Predicted Class: {predicted_class_name}, Actual Class: {class_name}")

            # Check if the prediction is correct
            if predicted_class_name == class_name:
                correct_predictions += 1

            total_predictions += 1

# Calculate and print the final accuracy
accuracy = (correct_predictions / total_predictions) * 100
print(f"Final Accuracy: {accuracy:.2f}%")
